In [87]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv2D, MaxPooling2D, Flatten, TimeDistributed
from keras.utils import to_categorical

from time_expector import TimeExpector
from notify import notify
from sys import stdout

In [89]:
from data_augmentor import data_augment, batchify
from time_expector import TimeExpector
from cache_generator import GESTURE_MAPPING

CACHE_FOLDER_PATH = "/Users/aref/dvs-dataset/Cached/"
DATASET_FOLDER_PATH = "/Users/aref/dvs-dataset/DvsGesture/"
    
nb_image_height = 64
nb_image_width  = 64
nb_inputs  = nb_image_height*nb_image_width

time_step = 1e-3
nb_steps  = 100
batch_size = 128
nb_epochs = 20
nb_labels = len(GESTURE_MAPPING)

tx = TimeExpector()

In [90]:
def load_data(trail):
    if trail.startswith('acc'):
        max_augmentation = 1
        augmentation = False
    else:
        max_augmentation = 3 if trail == 'train' else 1
        augmentation = True
    
    trail = trail.replace('acc_', '')
    return batchify(
        trail,
        DATASET_FOLDER_PATH,
        CACHE_FOLDER_PATH,
        condition_limit=['natural'],
        batch_size=batch_size,
        augmentation=augmentation,
        max_augmentation=max_augmentation
    )

In [91]:
def prog_print(items, max_items):
    percentage = 100*items/max_items
    items = int(items)
    max_items = int(max_items)
    
    stdout.write('\r')
    stdout.write('  %.2f%%' % percentage)
    stdout.write('  [=')
    if items < max_items:
        stdout.write('=' * (items-1))
        stdout.write('>')
        stdout.write('.' * (max_items - items))
    else:
        stdout.write('=' * (items))
    stdout.write(']  ')
    stdout.flush()

In [92]:
keras_batch_size = 16
model = Sequential()

model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'),
                          input_shape=(None, 64, 64, 1),
                          batch_size=keras_batch_size
                         ))

model.add(TimeDistributed(MaxPooling2D((2, 2))))

# model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
# model.add(TimeDistributed(MaxPooling2D((2, 2))))

# model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
# model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(LSTM(64,
          batch_size=keras_batch_size,
          stateful=True))

model.add(Dense(nb_labels, activation='softmax'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


# model.add(TimeDistributedConvolution2D(32, 5, 5, border_mode='same', input_shape=(nb_steps, 64, 64)))
# model.add(TimeDistributedMaxPooling2D(pool_size=(2, 2)))
# model.add(Activation('relu'))

# model.add(TimeDistributedFlatten())
# model.add(LSTM(256, return_sequences=True))

# model.add(Dense(nb_labels))
# model.add(Activation('softmax'))
# model.compile(loss='mse', optimizer='adam')

In [93]:
notify('*A-RNN* has started', mark='info')
try:
    for e in range(nb_epochs):
        tx.tick(nb_epochs - e)
        local_loss_train = []
        local_loss_test = []
        local_acc_train = []
        local_acc_test = []

        print 'Epoch %d/%d' % (e+1,nb_epochs)

        pc = 0
        for x_local, y_local in load_data('train'):
            prog_print(pc, 55)
            pc += .1

            x_local = np.reshape(x_local, (batch_size, nb_steps, nb_image_width, nb_image_height, 1))
            y_local = to_categorical(y_local)

            history_train = model.fit(
                x_local,
                y_local,
                batch_size=keras_batch_size,
                epochs=1,
                verbose=0
            )
            model.reset_states()
            
        for x_local, y_local in load_data('test'):
            prog_print(pc, 55)
            pc += .1
            x_local = np.reshape(x_local, (batch_size, nb_steps, nb_image_width, nb_image_height, 1))
            y_local = to_categorical(y_local)

            history_test = model.evaluate(
                x_local,
                y_local,
                batch_size=keras_batch_size,
                verbose=0
            )

        mean_loss_train = np.mean(history_train.history['loss'])
        mean_loss_test  = np.mean(history_test[0])
        print('- loss (train=%.5f , test=%.5f)' % (mean_loss_train, mean_loss_test))
        local_loss_train.append(mean_loss_train)
        local_loss_test.append(mean_loss_test)

        local_acc_train.append(np.mean(history_train.history['acc']))
        local_acc_train.append(np.mean(history_test[1]))
        tx.tock()
    notify('*A-RNN* finished!', mark='ok')
except KeyboardInterrupt:
    print '\n\n-- canceled --'
except:
    notify('*A-RNN* failed...', mark='error')
    raise

ℹ️ *A-RNN* has started
Epoch 1/20
  0.36%  [=>.......................................................]  

-- canceled --
